inheritance, association, aggregation (mũi tên hình thoi tắng), composition (mũi tên hình thoi đen)

# Code 
GAME Ô ĂN QUAN VIỆT NAM (Mandarin square capturing) cho 2 người chơi dựa vào code dưới đây: 

2.1. controls package:  

- board subpackage 

- player subpackage 

- Game class 

2.2. gui package: 
- BoardDrawer class: 
- Drawer.class: 
- GameCanvas.class: 
- PlayerDrawer.class: 

- HelpPage class: 
- MainMenu class: 
- MainWindow class: 
2.3 MainProgram class: 


Chi tiết: 
2.1. controls package:  
```Stone.java
package controls.board;

public class Stone {
	protected int value;
	
	public int getValue() {
		return this.value;
	}
}
```

```
SmallGem.java
package controls.board;

public class SmallGem extends Stone {
    public SmallGem() {
        this.value = 1;
    }
}
```

```BigGem.java

package controls.board;

public class BigGem extends Stone {
    public BigGem() {
        this.value = 5;
    }
}
```

```BoardCell.java
package controls.board;

import java.util.*;

public class BoardCell {
	private ArrayList<Stone> stonesInCell;
	
	public BoardCell(BigGem quan) {
		stonesInCell = new ArrayList<Stone>();
		this.stonesInCell.add(quan);
	}

	public BoardCell(SmallGem dan) {
		stonesInCell = new ArrayList<Stone>();
		for(int i=0; i<5; i++) {
			this.stonesInCell.add(dan);
		}
	}

	public ArrayList<Stone> getStonesInCell() {
		return this.stonesInCell;
	}
	
	public int getNumberOfStones() {
		return this.stonesInCell.size();
	}
	
	public int getPoint() {
		int point = 0;
		for(Stone s: this.stonesInCell) {
			point += s.getValue();
		}
		return point;
	}
}
```

```Board.java
package controls.board;

public class Board {
	private BoardCell[] cells;
	
	public Board() {
		cells = new BoardCell[12];
		for(int i = 0; i<12; i++) {
			if ((i == 5) || (i==11)) {
				this.cells[i] = new BoardCell(new BigGem());
			}
			else {
				this.cells[i] = new BoardCell(new SmallGem());	
			}
		}
	}
	
	public void reset() {
		cells = new BoardCell[12];
		for(int i = 0; i<12; i++) {
			if ((i == 5) || (i==11)) {
				this.cells[i] = new BoardCell(new BigGem());
			}
			else {
				this.cells[i] = new BoardCell(new SmallGem());	
			}
		}
	}
	
	public BoardCell[] getCells() {
		return this.cells;
	}
	
	public boolean gameEnd() {
		return (cells[5].getNumberOfStones() == 0 && cells[11].getNumberOfStones() == 0);
	}
}
```

```
Player.java

package controls.player;

import java.util.*;
import java.util.concurrent.*;
import java.lang.Math;

import controls.board.*;

public class Player {
	private ArrayList<Stone> inHand;
	private ArrayList<Stone> taken;
	private int playerId;
	private int dir = -1;
	private int curIndex = -1;
	private int penalty = 0;
	
	public Player(int id) {
		this.inHand = new ArrayList<Stone>();
		this.taken = new ArrayList<Stone>();
		this.playerId = id;
	}
	
	public void reset() {
		this.inHand = new ArrayList<Stone>();
		this.taken = new ArrayList<Stone>();
		curIndex = -1;
		penalty = 0;
	}
	
	public void makeMove(Board b) {
		if(isTurn()) {
			int nextIndex = Math.floorMod(this.curIndex+dir, 12);
			int afterIndex = Math.floorMod(this.curIndex+2*dir, 12);
			BoardCell cur = b.getCells()[curIndex];
			BoardCell next = b.getCells()[nextIndex];
			BoardCell after = b.getCells()[afterIndex];
			int mc = moveCase(b, cur, next, after);
			switch(mc) {
			case 0:
				releaseStone(cur);
				break;
			case 1:
				pickupStones(cur);
				break;
			case 2:
				takeStonesInNext(next, mc==2);
				break;
			case 3:
				takeStonesInNext(next, mc==2);
				break;
			case 4:
				this.curIndex = -1;
				break;
			}
		}
		try {
			TimeUnit.MILLISECONDS.sleep(300);
		}
		catch(Exception e) {
			e.printStackTrace();
		}
	}
	
	public boolean isTurn() {
		return (this.curIndex>=0);
	}
	
	public void raiSoi(Board b) {
		if(playerId == 1) {
			for(int i=0; i<5; i++) {
				b.getCells()[i].getStonesInCell().add(new Stone(false));
			}
		}
		if(playerId == 2) {
			for(int i=6; i<11; i++) {
				b.getCells()[i].getStonesInCell().add(new Stone(false));
			}
		}
		this.penalty ++;
	}
	
	public void pickupStones(BoardCell bc) {
		ArrayList<Stone> cur = bc.getStonesInCell();
		if(bc.isOQuan()) {
			this.curIndex = -1;
		}
		else {
			this.inHand.addAll(cur);
			cur.clear();
			this.curIndex = Math.floorMod((curIndex+dir), 12);
		}
	}
	
	public void releaseStone(BoardCell bc) {
		//System.out.println(this.inHand.size());
		if(this.inHand.size()>0) {
			ArrayList<Stone> cur = bc.getStonesInCell();
			cur.add(this.inHand.get(this.inHand.size()-1));
			this.inHand.remove(this.inHand.size()-1);
			//System.out.println(this.inHand.size());
			this.curIndex = Math.floorMod(this.curIndex+dir, 12);
		}
	}
	
	public void takeStonesInNext(BoardCell next, boolean endTurn) {
		ArrayList<Stone> stones = next.getStonesInCell();
		this.taken.addAll(stones);
		stones.clear();
		if(endTurn) {
			this.curIndex = -1;
		}
		else {
			this.curIndex = Math.floorMod(this.curIndex+2*dir, 12);
		}
	}
	
	public int moveCase(Board b, BoardCell cur, BoardCell next, BoardCell after) { 
		int ret = 0;
		
		if(this.inHand.size() > 0) {
			ret = 0;//Release 1 stone to cell
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones()>0) {
			ret = 1;//Pickup all stones in cell
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() > 0 && after.getNumberOfStones()>0) {
			ret = 2;//Eat all stones in next cell and end turn
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() > 0 && after.getNumberOfStones()==0) {
			ret = 3;//Eat all stones in next cell and continue
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() == 0) {
			ret = 4;//End turn
		}
		
		return ret;
	}
	
	public int getPoint() {
		int ret = 0;
		for(Stone s: this.taken) {
			ret += s.getValue();
		}
		return ret - this.penalty*5;
	}
	
	public int getNumberTakenStones() {
		return this.taken.size();
	}
	
	public void setCurIndex(int curIndex) {
		this.curIndex = curIndex;
	}
	
	public void setDir(int dir) {
		this.dir = dir;
	}
	
	public boolean isValidMove(int ci) {
		boolean ret = true;
		switch(this.playerId) {
		case 1:
			if(ci<0 || ci>4) {
				ret = false;
			}
			
			break;
		case 2:
			if(ci<6 || ci>10) {
				ret = false;
			}
		}
		return ret;
	}
	
	public void moveSetup(int ci, int d) {
		if(isValidMove(ci)) {
			this.curIndex = ci;
			this.dir = d;
		}
	}
	
	public int getPlayerId() {
		return this.playerId;
	}
}

```

```
Game.java

package controls;

import controls.board.*;
import controls.player.Player;

public class Game {
	private Board myBoard;
	private Player player1;
	private Player player2;
	private boolean isP1Turn;
	private boolean waitMove;
	
	public Game() {
		myBoard = new Board();
		player1 = new Player(1);
		player2 = new Player(2);
		isP1Turn = false;
		waitMove = true;
	}
	
	public void restart() {
		myBoard.reset();;
		player1.reset();;
		player2.reset();;
		isP1Turn = false;
		waitMove = true;
	}
	
	public void playGame() {
		if(!waitMove && !myBoard.gameEnd()) {
			if(isP1Turn) {
				player1.makeMove(myBoard);
				if(player1.isTurn() != isP1Turn) {
					waitMove = true;
					isP1Turn = player1.isTurn();
				}
			}
			else {
				player2.makeMove(myBoard);
				if(player2.isTurn() == isP1Turn) {
					waitMove = true;
					isP1Turn = !player2.isTurn();
				}
			}
		}
		if(waitMove && !myBoard.gameEnd() && raiSoi()>0) {
			if(raiSoi() == 1 && isP1Turn) {
				player1.raiSoi(myBoard);
			}
			if(raiSoi() == 2 && !isP1Turn) {
				player2.raiSoi(myBoard);
			}
		}
	}
	
	public boolean waitingForMove() {
		return waitMove;
	}
	
	public boolean isP1Turn() {
		return isP1Turn;
	}
	
	public void setMove(int ci, int d) {
		if(isValidMove(ci)) {
			if(isP1Turn) {
				this.player1.moveSetup(ci, d);
			}
			else {
				this.player2.moveSetup(ci, d);
			}
			this.waitMove = false;
		}
	}
	
	public boolean isValidMove(int oc) {
		boolean ret = false;
		if(this.isP1Turn() && oc<5 && oc>=0 && myBoard.getCells()[oc].getNumberOfStones()>0) {
			ret = true;
		}
		if(!this.isP1Turn() && oc<11 && oc>5 && myBoard.getCells()[oc].getNumberOfStones()>0) {
			ret = true;
		}
		return (ret && this.waitingForMove());
	}
	
	public int raiSoi() {
		int ret = 0;
		boolean p1 = false;
		boolean p2 = false;
		
		for(int i = 0; i<5; i++) {
			if(myBoard.getCells()[i].getNumberOfStones() > 0) {
				p1 = true;
			}
		}
		
		for(int i = 6; i<11; i++) {
			if(myBoard.getCells()[i].getNumberOfStones() > 0) {
				p2 = true;
			}
		}
		
		if(!p1) {
			ret = 1;
		}
		
		if(!p2) {
			ret = 2;
		}
		
		return ret;
	}
	
	public Board getBoard() {
		return this.myBoard;
	}
	
	public Player getPlayer1() {
		return this.player1;
	}
	
	public Player getPlayer2() {
		return this.player2;
	}
	
	public boolean gameEnd() {
		return this.myBoard.gameEnd();
	}
	
	public int getWinner() {
		int ret = -1;
		if(player1.getPoint()>player2.getPoint()) {
			ret = 1;
		}
		else if(player2.getPoint()>player1.getPoint()) {
			ret = 2;
		}
		else {
			ret = 3;
		}
		return ret;
	}
}

```

2.2. gui package: 
```
BoardDrawer.java

package gui;

import javax.swing.*;
import java.awt.*;
import java.awt.event.*;
import java.lang.Math;
import java.util.*;

import controls.Game;
import controls.board.*;

public class BoardDrawer extends Drawer {
	private Board myBoard;
	private Game myGame;
	private int cellSize = 100;
	private int danRadius = 5;
	private int quanRadius = 15;
	
	public BoardDrawer(Board myBoard, JPanel cp) {
		super(cp);
		this.myBoard = myBoard;
	}
	
	public BoardDrawer(Board myBoard, JPanel cp, int size, int dan, int quan) {
		super(cp);
		
		this.myBoard = myBoard;
		
		this.cellSize = size;
		this.danRadius = dan;
		this.quanRadius = quan;
	}
	
	public BoardDrawer(Game myGame, JPanel cp) {
		super(cp);
		this.myGame = myGame;
		this.myBoard = myGame.getBoard();
	}
	
	public BoardDrawer(Game myGame, JPanel cp, int size, int dan, int quan) {
		super(cp);
		this.myGame = myGame;
		this.myBoard = myGame.getBoard();
		
		this.cellSize = size;
		this.danRadius = dan;
		this.quanRadius = quan;
	}
	
	public void draw() {
		Graphics2D graphic2d = (Graphics2D) this.myG;
		setup(graphic2d);
		drawBoardLines(graphic2d);
		drawStones(graphic2d);
		if(this.myGame.isValidMove(onCellControl())) {
			highlightCell(graphic2d);
		}
		//graphic2d.drawOval(mouseX, mouseY, quanRadius, quanRadius);
	}
	
	public void setup(Graphics2D g) {
		g.clearRect(0, 0, width, height);
		g.setStroke(new BasicStroke(5));
		g.setFont(new Font("SansSerif", Font.BOLD, 20));
	}
	
	private void drawBoardLines(Graphics2D g) {
		g.setColor(Color.BLACK);
		for(int i = 0; i<10; i++) {
			int x = getWidth()/2 - (5*this.cellSize)/2 + (i%5)*this.cellSize;
			int y = getHeight()/2 - this.cellSize + (i/5)*this.cellSize;
			g.drawRect(x, y, this.cellSize, this.cellSize);
		}
		g.drawArc((getWidth()/2-(7*this.cellSize)/2), (getHeight()/2-this.cellSize), 2*this.cellSize, 2*this.cellSize, 90, 180);
		g.drawArc((getWidth()/2+(3*this.cellSize)/2), (getHeight()/2-this.cellSize), 2*this.cellSize, 2*this.cellSize, -90, 180);
	}
	
	private void drawStones(Graphics2D g) {
		BoardCell[] cells = this.myBoard.getCells();
		for(int i = 0; i<12; i++) {
			if(!cells[i].isOQuan()) {
				drawInDan(g, i, cells[i]);
			}
			else {
				drawInQuan(g, i, cells[i]);
			}
		}
	}
	
	private void drawInDan(Graphics2D g, int ind, BoardCell cell) {
		g.setColor(Color.BLACK);
		ArrayList<Stone> stones = cell.getStonesInCell();
		int ord = (ind<5)?ind:(5+(9-(ind-1)));
		int num = stones.size();
		int side = (int)Math.ceil(Math.sqrt(num));
		int d = (int)(this.cellSize*0.9/(side+1));
		int centerX = getWidth()/2 - 2*this.cellSize + (ord%5)*this.cellSize;
		int centerY = getHeight()/2 - this.cellSize/2 + (ord/5)*this.cellSize;
		for(int i=0; i<stones.size(); i++) {
			Stone s = stones.get(i);
			int r = this.danRadius;
			if(s.isQuan()) {
				r = this.quanRadius;
			}
			g.drawOval((int)(centerX + ((i%side)-(side*0.5-0.5))*d) - r/2, (int)(centerY - ((i/side)-(side*0.5-0.5))*d) - r/2, r, r);
		}
		g.setColor(new Color(150, 150, 150));
		g.drawString(cell.getPoint()+"", centerX - this.cellSize*9/20, centerY + this.cellSize*9/20);
	}
	
	private void drawInQuan(Graphics2D g, int ind, BoardCell cell) {
		g.setColor(Color.BLACK);
		ArrayList<Stone> stones = cell.getStonesInCell();
		int ord = (ind > 5) ? (-1):(1);
		int num = stones.size();
		int side = (int)Math.ceil(Math.sqrt(num/1.5));
		int d = (int)(this.cellSize*0.8/(side+1));
		int centerX = (int)(getWidth()/2 + ord*(2.5*this.cellSize + 0.4*this.cellSize));
		int centerY = getHeight()/2;
		for(int i=0; i<num; i++) {
			Stone s = stones.get(i);
			int r = this.danRadius;
			if(s.isQuan()) {
				r = this.quanRadius;
			}
			g.drawOval((int)(centerX - ord*((i%side)-(side*0.5-0.5))*d) - r/2, (int)(centerY - ((i/side)-(side*0.75-0.5))*d) - r/2, r, r);
		}
		g.setColor(new Color(150, 150, 150));
		g.drawString(cell.getPoint()+"", centerX - ord*this.cellSize/5 - (cell.getPoint()+"").length()*5, centerY + ord*this.cellSize*8/10 + 10);
	}
	
	public int onCell() {
		int x = (this.mouseX - width/2 + (5*this.cellSize)/2)/this.cellSize;
		int y = (this.mouseY - height/2 + this.cellSize)/this.cellSize;
		x = (x>=0 && x<5 && (this.mouseX - width/2 + (5*this.cellSize)/2)>0)?x:(-1);
		y = (y>=0 && y<2 && (this.mouseY - height/2 + this.cellSize)>0)?y:(-1);
		int i = (x>=0 && y>=0)?(x+y*5):(-1);
		return i;
	}
	
	public int onCellControl() {
		int ind = onCell();
		return (ind<5)?ind:(5+(9-(ind-1)));
	}
	
	public int getDir() {
		int ret = 1;
		int ind = onCell();
		int centerX = getWidth()/2 - 2*this.cellSize + (ind%5)*this.cellSize;
		if(mouseX>centerX == (ind>=0 && ind<5)) {
			ret = 1;
		}
		else{
			ret = -1;
		}
		return ret;
	}
	
	public void highlightCell(Graphics2D g) {
		if(this.myGame.isValidMove(onCellControl())) {
			int i = onCell();
			if(i>=0) {
				int x = getWidth()/2 - (5*this.cellSize)/2 + (i%5)*this.cellSize;
				int y = getHeight()/2 - this.cellSize + (i/5)*this.cellSize;
				int cx = x + this.cellSize/2;
				int cy = y + this.cellSize/2;
				Polygon leftArrow = new Polygon(new int[] {x+10, x+10, x-10}, new int[] {cy-10, cy+10, cy}, 3);
				Polygon rightArrow = new Polygon(new int[] {x+cellSize-10, x+cellSize-10, x+cellSize+10}, new int[] {cy-10, cy+10, cy}, 3);
				g.setColor(Color.BLUE);
				g.drawRect(x, y, this.cellSize, this.cellSize);
				g.setColor(Color.GREEN);
				if(mouseX>cx) {
					g.drawPolygon(rightArrow);
					g.fillPolygon(rightArrow);
				}
				else {
					g.drawPolygon(leftArrow);
					g.fillPolygon(leftArrow);
				}
			}
		}
	}
	
	public void mouseClicked(MouseEvent e) {
		this.myGame.setMove(onCellControl(), getDir());
	}
}

```
```
Drawer.java

package gui;

import java.awt.*;
import java.awt.event.*;
import javax.swing.*;

public class Drawer implements MouseListener, MouseMotionListener{
	protected Graphics myG;
	protected JPanel myPa;
	protected int width;
	protected int height;
	protected int mouseX;
	protected int mouseY;
	protected boolean mouseClicked = false;
	
	protected Drawer(JPanel myPa) {
		this.myPa = myPa;
		this.myPa.addMouseListener(this);
		this.myPa.addMouseMotionListener(this);
	}
	
	public void mousePressed() {
		if(mouseClicked) {
			return;
		}
		mouseClicked = false;
	}
	
	public void setMyG(Graphics g) {
		this.myG = g;
		Rectangle r = g.getClipBounds();
		this.height = (int)r.getHeight();
		this.width = (int)r.getWidth();
	}
	
	public int getWidth() {
		return this.width;
	}
	
	public int getHeight() {
		return this.height;
	}

	@Override
	public void mouseDragged(MouseEvent e) {}

	@Override
	public void mouseMoved(MouseEvent e) {
		this.mouseX = e.getX();
		this.mouseY = e.getY();
	}

	@Override
	public void mouseClicked(MouseEvent e) {}

	@Override
	public void mousePressed(MouseEvent e) {}

	@Override
	public void mouseReleased(MouseEvent e) {}

	@Override
	public void mouseEntered(MouseEvent e) {}

	@Override
	public void mouseExited(MouseEvent e) {}
	
}

```
GameCanvas.java

package gui;

import controls.board.*;
import controls.*;
import java.awt.*;
import java.awt.event.*;
import javax.swing.*;

public class GameCanvas extends JPanel{
	private BoardDrawer myBD;
	private PlayerDrawer myPD;
	private JButton back;
	private MainWindow pa;
	
	public GameCanvas(Board myBoard) {
		this.myBD = new BoardDrawer(myBoard, this);
	}
	public GameCanvas(Game myGame, MainWindow pa) {
		this.pa = pa;
		this.myBD = new BoardDrawer(myGame, this);
		this.myPD = new PlayerDrawer(myGame, this);
		
		ActionListener goBack = new ActionListener() {
			@Override
			public void actionPerformed(ActionEvent e) {
				int confirmed = JOptionPane.showConfirmDialog(null, "Are you sure you want to quit?", "Confirm quit", JOptionPane.YES_NO_OPTION);
				if (confirmed == JOptionPane.YES_OPTION) {
					pa.backToMain();
				}
			}
		};
		
		this.back = new JButton("Back");
		this.back.setLocation(getWidth() - this.back.getWidth(), getHeight()-this.back.getHeight());
		this.back.addActionListener(goBack);
		
		add(this.back);
	}
	
	@Override
	protected void paintComponent(Graphics g) {
		super.paintComponent(g);
		
		((Graphics2D)g).setBackground(new Color(242, 242, 242));
		
		this.back.setLocation(getWidth() - this.back.getWidth(), getHeight()-this.back.getHeight());
		
		this.myBD.setMyG(g);
		this.myPD.setMyG(g);
		
		this.myBD.draw();
		this.myPD.draw();
	}
}

```
PlayerDrawer.java

package gui;

import java.awt.*;
import javax.swing.*;

import controls.*;
import controls.player.Player;

public class PlayerDrawer extends Drawer{
	private Game myGame;
	private int d = 150;
	
	public PlayerDrawer(Game myGame, JPanel cp) {
		super(cp);
		
		this.myGame = myGame;
	}
	
	public void draw() {
		Graphics2D g2d = (Graphics2D) this.myG; 
		setup(g2d);
		drawPlayers(g2d);
	}
	
	public void setup(Graphics2D g) {
		g.setStroke(new BasicStroke(5));
		g.setColor(Color.BLACK);
		g.setFont(new Font("SansSerif", Font.BOLD, 30));
	}
	
	public void drawPlayers(Graphics2D g) {
		FontMetrics metrics = g.getFontMetrics(g.getFont());
		String p1 = this.myGame.getPlayer1().getPoint()+"";
		String p2 = this.myGame.getPlayer2().getPoint()+"";
		
		g.drawRect(width/2 - d/2, 0, d, d*4/5);
		g.drawString("Player 2", width/2 - metrics.stringWidth("Player 2")/2, d/5+metrics.getHeight()/4);
		g.drawString(p1, width/2 - metrics.stringWidth(p1)/2, d/2+metrics.getHeight()/4);
		g.drawRect(width/2 - d/2, height - d*4/5, d, d*4/5);
		g.drawString("Player 1", width/2 - metrics.stringWidth("Player 1")/2, height-d/5+metrics.getHeight()/4);
		g.drawString(p2, width/2 - metrics.stringWidth(p2)/2, height-d/2+metrics.getHeight()/4);
		
		g.setColor(Color.MAGENTA);
		g.setFont(new Font("SansSerif", Font.BOLD, 20));
		if(!this.myGame.gameEnd()) {
			if(this.myGame.isP1Turn()) {
				g.drawRect(width/2 - d/2, 0, d, d*4/5);
				g.drawString("Player 2's turn", d/2, d/5);
			}
			else {
				g.drawRect(width/2 - d/2, height - d*4/5, d, d*4/5);
				g.drawString("Player 1's turn", d/2, d/5);
			}
		}
		else {
			int win = 3 - this.myGame.getWinner();
			if(win>0) {
				g.drawString("Player " + win + " win", d/2, d/5);
			}
			else {
				g.drawString("Tie", d/2, d/5);
			}
		}
		
	}
}


```
```
HelpPage.java

package gui;

import java.awt.*;
import java.awt.event.*;
import javax.swing.*;
import javax.swing.border.*;

public class HelpPage extends JPanel{
	private JLabel content;
	private JButton back;
	private MainWindow pa;
	
	public HelpPage(MainWindow m) {
		this.pa = m;
		this.content = new JLabel("<html>"
				+ "<h1>How to play</h1>"
				+ "<p>"
				+ "Hover the cursor over one of five square in your side.<br>"
				+ "The square will turn blue if it's a valid move.<br>"
				+ "An arrow will appear depends on whether the cursor is to the left or the right of the square, indicating the direction of the move.<br>"
				+ "You click on the square to make your move.<br>"
				+ "The game ends when both big cells are emptied.<br>"
				+ "</p>"
				+ "<h1>Hướng dẫn chơi game</h1>"
				+ "<p>"
				+ "Đưa chuột lên một trong năm ô thuộc về bên của bạn.<br>"
				+ "Ô đó sẽ chuyển màu xanh dương.<br>"
				+ "Nếu bạn đưa chuột về bên phải ô sẽ hiện lên ▶, nếu đưa chuột về bên trái ô sẽ hiện lên ◀, tương ứng với chiều rải quân.<br>"
				+ "Bạn click chuột để thực hiện nước đi của mình.<br>"
				+ "Trò chơi kết thúc khi hai ô quan đã được ăn hết.<br>"
				+ "</html>");
		this.content.setBorder(new EmptyBorder(50, 50, 50, 50));
		
		this.back = new JButton("Back");
		ActionListener goBack = new ActionListener() {
			@Override
			public void actionPerformed(ActionEvent e) {
				pa.backToMain();
				//System.out.println("2");
			}
		};
		this.back.addActionListener(goBack);
		
		setLayout(new BoxLayout(this, BoxLayout.Y_AXIS));
		this.content.setAlignmentX(CENTER_ALIGNMENT);
		this.back.setAlignmentX(CENTER_ALIGNMENT);
		
		add(this.content);
		add(this.back);
	}
}

```

```MainMenu.java
package gui;

import java.awt.*;
import java.awt.event.*;
import javax.swing.*;

public class MainMenu extends JPanel{
	private JButton start;
	private JButton help;
	private JButton exit;
	private JLabel title;
	private MainWindow pa;
	
	public MainMenu(MainWindow pa) {
		this.pa = pa;
		title = new JLabel("Ô Ăn Quan", JLabel.CENTER);
		title.setFont(new Font("SansSerif", Font.PLAIN, 50));
		
		start = new JButton("New game");
		help = new JButton("Help");
		exit = new JButton("Exit");
		
		ActionListener startGame = new ActionListener() {
			@Override
			public void actionPerformed(ActionEvent e) {
				pa.newGame();
			}
		};
		
		ActionListener openHelp = new ActionListener() {
			@Override
			public void actionPerformed(ActionEvent e) {
				pa.showHelp();
			}
		};
		
		ActionListener exitGame = new ActionListener() {
			@Override
			public void actionPerformed(ActionEvent e) {
				pa.exitGame();
			}
		};
		
		start.addActionListener(startGame);
		help.addActionListener(openHelp);
		exit.addActionListener(exitGame);
		
		setLayout(new GridBagLayout());
		GridBagConstraints c = new GridBagConstraints();
		
		c.fill = GridBagConstraints.HORIZONTAL;
		c.gridx = 0;
		c.gridy = 0;
		c.gridwidth = 4;
		c.ipady = 50;
		add(title, c);
		
		c.fill = GridBagConstraints.HORIZONTAL;
		c.gridx = 0;
		c.gridy = 1;
		c.gridwidth = 1;
		c.ipady = 0;
		add(start, c);
		
		c.gridx = 1;
		c.gridy = 1;
		add(help, c);
		
		c.gridx = 2;
		c.gridy = 1;
		add(exit, c);
	}
}

```
```
MainWindow.java
package gui;

import javax.swing.*;
import javax.swing.JOptionPane;
import java.awt.*;

import controls.*;

public class MainWindow extends JFrame{
	private GameCanvas myGC;
	private MainMenu myMenu;
	private HelpPage myHelp;
	private int mode;
	private Game myGame;
	private CardLayout c;
	//private JMenuBar myMB;
	
	public MainWindow(Game g) {
		this.myGame = g;
		this.mode = 1;
		myGC = new GameCanvas(this.myGame, this);
		myMenu = new MainMenu(this);
		myHelp = new HelpPage(this);
		c = new CardLayout();
		setLayout(c);
		
		add("Menu", myMenu);
		add("GC", myGC);
		add("Help", myHelp);
		
		setVisible(true);
		setSize(800, 600);
		setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
		setTitle("Ô Ăn Quan");
		setLocationRelativeTo(null);
	}
	
	public void playGame() {
		this.myGame.playGame();
	}
	
	public void newGame() {
		this.myGame.restart();
		c.show(getContentPane(), "GC");
		mode = 2;
	}
	
	public void showHelp() {
		c.show(getContentPane(), "Help");
		mode = 3;
	}
	
	public void backToMain() {
		c.show(getContentPane(), "Menu");
		mode = 4;
	}
	
	public void exitGame() {
		int confirmed = JOptionPane.showConfirmDialog(null, "Are you sure you want to quit?", "Exit messge", JOptionPane.YES_NO_OPTION);
		if(confirmed == JOptionPane.YES_OPTION) {
			System.exit(0);
		}
	}
	
	public void redraw() {
		this.myGC.repaint();
	}
	
	public int getMode() {
		return mode;
	}
}

```

```MainProgram.java

import controls.Game;
import gui.MainWindow;

public class MainProgram {
	public static void main(String[] args) {
		Game myGame = new Game();
		MainWindow myWindow = new MainWindow(myGame);
		
		while(true) {
			while(myWindow.getMode() == 2) {
				myWindow.redraw();
				myWindow.playGame();
			}
			for(int i=0; i<5000; i++) {}
		}
	}
}

```




# Use Case Diagram and Class diagram
                           +-------------------+
                           |       Game        |
                           +-------------------+
                           | - myBoard: Board  |
                           | - player1: Player |
                           | - player2: Player |
                           | - isP1Turn: bool  |
                           | - waitMove: bool  |
                           +-------------------+
                           | + restart():void       |
                           | + playGame():void      |
                           | + waitingForMove():bool|
                           | + isP1Turn():bool      |
                           | + setMove(ci, d):void |
                           | + isValidMove(oc):bool|
                           | + raiSoi(): int       |
                           | + getBoard():Board     |
                           | + getPlayer1():Player    |
                           | + getPlayer2():Player    |
                           | + gameEnd():bool       |
                           | + getWinner():int    |
                           +-------------------+

                                 ^
                                 |
                                 |
                +-------------------------------------+
                |                Board                |
                +-------------------------------------+
                | - cells: BoardCell[]                |
                +-------------------------------------+
                | + reset():void                            |
                | + getCells(): BoardCell[]            |
                | + gameEnd(): bool                    |
                +-------------------------------------+

                                 ^
                                 |
                                 |
         +-----------------------------------+
         |            BoardCell             |
         +-----------------------------------+
         | - stonesInCell: ArrayList<Stone> |
         | - isOQuan: bool                   |
         +-----------------------------------+
         | + getStonesInCell(): ArrayList<Stone> |
         | + getNumberOfStones(): int              |
         | + getPoint(): int                       |
         | + isOQuan(): bool                        |
         +-----------------------------------+

                                 ^
                                 |
                                 |
         +-----------------------------------+
         |              Stone                |
         +-----------------------------------+
         | - value: int                      |
         | - isQuan: bool                    |
         +-----------------------------------+
         | + getValue(): int                 |
         | + isQuan(): bool                  |
         +-----------------------------------+

                                 ^
                                 |
                                 |
         +-----------------------------------+
         |             Player                |
         +-----------------------------------+
         | - inHand: ArrayList<Stone>        |
         | - taken: ArrayList<Stone>         |
         | - playerId: int                   |
         | - dir: int=-1                     |
         | - curIndex: int=-1                   |
         | - penalty: int=0                    |
         +-----------------------------------+
         | + reset():void                         |
         | + makeMove(b: Board):void             |
         | + isTurn(): bool                  |
         | + raiSoi(b: Board):void                |
         | + pickupStones(bc: BoardCell):void     |
         | + releaseStone(bc: BoardCell):void     |
         | + takeStonesInNext(next: BoardCell, endTurn: bool) |
         | + moveCase(b: Board, cur: BoardCell, next: BoardCell, after: BoardCell): int |
         | + getPoint(): int                 |
         | + getNumberTakenStones(): int     |
         | + setCurIndex(curIndex: int)      |
         | + setDir(dir: int)                |
         | + isValidMove(ci: int): bool      |
         | + moveSetup(ci: int, d: int)      |
         | + getPlayerId(): int              |
         +-----------------------------------+


## 2.3. Use cases

In this use case, the actor is the player. As the game requires two players to play, two players are represented as two children inheriting the parent (player). The player has 4 use cases: Play the game, view Help, check game status and quit the game.

- Play the game: The user can start a new game instance from the main menu screen. Then, the application displays the game board; the players play the game in accordance with the rules stated until the game ends.

- Check game status: The players can easily check how the game progresses with real-time score displaying for each player.

- View Help: The user can choose to view the help guide for the game by choosing the Help button from the main menu screen. The guide is available in both English and Vietnamese.

- Quit the game: The player has two ways to exit the game: in the main menu screen (the Exit button) or during the game (the Quit button on the game board). In either case, a confirmation dialog will be displayed to confirm the user's intention. The user can click OK to confirm their intention, or click Cancel should the user changes their minds.
It's worth noting that choosing Quit from the game board will only take the user back to the main menu. To completely exit the program, the user should use the Exit button on the main menu screen.


# 3. Design: 


## 3.1 General class diagram and classes relationship

### General class diagram:
There is a clear distinction between two packages of the program: the controls package - which represents the inner working parts of the program; and the gui package - which contains the Graphical User Interface for the program. There is also a MainProgram class which is our entry to the program.

The controls package contains two subpackages: player, board and the Game class which is the central point of the game.

### Relationships:
Relationships: For the controls package, The abstract Stone class has two child classes inherited from it: BigGem and SmallGem; in turn, this Stone class are aggregated by the abstract BoardCell class (with two child classes: SmallBoardCell and BigBoardCell). The Board class are then aggregated by the BoardCell class, which will become the game board for players to interact with. The Player class is associated with Board and BoardCell class, and has a Composition relationship with Stone class. The Game class has a Composition relationship with both Player and Board.

For the gui package, MainWindow class has a Composition relationship with MainMenu, HelpPanel and GamePanel. The GamePanel class has a Composition relationship with PlayerDrawer and BoardDrawer, both of which are inherited from Drawer. 

The MainProgram class is aggregated by Game and is composited by MainWindow.

Further details on the classes will be discussed in the following sections.






## 3.2 Detailed class diagram and classes relationship

### Package controls

#### Subpackage board

- Stone class: the basic class to create stones in the game. This class has a property `value` to store the value of the rock and a method `getValue()` to return the value of the rock.
- SmallGem class: class inheriting from Stone class to create small stones. This class initializes the value of stone to 1.
- BigGem class: class inheriting from Stone class to create large stones. This class initializes the value of stone to 5.
- BoardCell class: class to create tiles on the chessboard. This class has a property `stonesInCell` to store a list of stones in the tile and a method `getStonesInCell()` to return a list of stones in the tile, a method `getNumberOfStones()` to return the number of stones in the tile, and a method `getPoint()` to calculate the total score of the stones in the tile.
- Board class: class to create chessboard. This class has the property `cells` to store a list of tiles on the chessboard and the method `getCell()` to return the tile at a specific position, the method `move()` to move the stones from one tile to another, and the method `getQuanPlayer()` to return the current player.

#### Subpackage player
- Player class: class to create players. This class has a property `name` to store the player's name and a method `getName()` to return the player's name.### Package guide- BoardDrawer class: class to draw chessboard. This class has a `draw()` method to draw a chessboard and related objects.
- Drawer class: class to draw. This is an abstract class and has no concrete methods.- GameCanvas class: class for drawing chessboards and related objects. This class has a `paintComponent()` method to draw objects.
- PlayerDrawer class: class to draw players. This class has a `draw()` method to draw player information.
- HelpPage class: class to display the help page. This class has a `display()` method to display the help page.- MainMenu class: class to display the main menu. This class has a `display()` method to display the menu and a `getMode()` method to return the selected game mode.
- MainWindow class: class to display the main interface of the game. This class has a property `game` to save the state of the game, a method `redraw()` to redraw the interface, a method `playGame()` to play the game, and other methods for handling events.

### MainProgram class
- MainProgram class: the main class of the game, containing the `main()` method to initialize the game and interface. In the `main()` method, create a `Game` object and a `MainWindow` object, and then use an infinite loop to render the interface and play the game. In the loop, you use the `redraw()` method to redraw the interface and the `playGame()` method to play the game.



+-----------------+       +----------------+
|     Board       |       |      Game      |
+-----------------+       +----------------+
| - size: int     |       | - board: Board |
| - cells: Cell[] |       +----------------+
+-----------------+       | +getBoard(): Board |
| +getSize(): int |       | +isValidMove(): boolean |
| +getCells(): Cell[]|    +----------------+
| +getCell(int, int): Cell |
+-----------------+

        ^
        |
        |
+-------+-------+
|               |
|    Cell       |
|               |
+---------------+
| - x: int      |
| - y: int      |
| - stone: Stone|
+---------------+
| +getX(): int  |
| +getY(): int  |
| +getStone(): Stone|
| +setStone(Stone): void|
+---------------+

        ^
        |
        |
+-------+-------+
|               |
|    Stone      |
|               |
+---------------+
| - color: Color|
+---------------+
| +getColor(): Color|
+---------------+

        ^
        |
        |
+-------+-------+
|               |
|    Drawer     |
|               |
+---------------+
| - myG: Graphics|
| - canvas: JPanel|
+---------------+
| +Drawer(JPanel): void|
| +getMyG(): Graphics|
| +getWidth(): int|
| +getHeight(): int|
| +onCellControl(): Cell|
+---------------+

        ^
        |
        |
+-------+-------+
|               |
| BoardDrawer   |
|               |
+---------------+
| - myBoard: Board|
| - myGame: Game  |
| - cellSize: int |
| - danRadius: int|
| - quanRadius: int|
+---------------+
| +BoardDrawer(Board, JPanel): void|
| +BoardDrawer(Board, JPanel, int, int, int): void|
| +BoardDrawer(Game, JPanel): void|
| +BoardDrawer(Game, JPanel, int, int, int): void|
| +draw(): void  |
| +setup(Graphics2D): void|
| -drawBoardLines(Graphics2D): void|
| -drawStones(Graphics2D): void|
| -highlightCell(Graphics2D): void|
+---------------+

        ^
        |
        |
+-------+-------+
|               |
|  GameCanvas   |
|               |
+---------------+
| - myGame: Game  |
+---------------+
| +GameCanvas(Game): void|
+---------------+


# Some questions

1. Describe the relationships between classes: 

- isQuan and Stone: inheritance
- isDan and Stone: inheritance

- BoardCell and Stone: composition.
(The BoardCell class has an ArrayList called stonesInCell, which contains Stone objects.
 Without the Stone class, the BoardCell class cannot function). 
- Board and BoardCell: composition.
- Game and Board: composition.
- Player and Stone: composition.

- Game and Player: composition.

- Board and Player: association.
- BoardCell and Player: association. 

2. Implementation of some important methods: 

Encapsulation (điều chỉnh giới hạn quyền truy câpj)
là một nguyên tắc trong lập trình hướng đối tượng (OOP) mà giới hạn việc truy cập trực tiếp đến dữ liệu và phương thức của một đối tượng từ bên ngoài. Nó đảm bảo tính riêng tư và bảo mật của dữ liệu, cũng như cung cấp một cách giao tiếp chỉ thông qua các phương thức công khai của đối tượng.

Trong đoạn code trên, tính encapsulation được thể hiện như sau:

Các lớp trong package controls (như Player, Board, Game) sử dụng tính encapsulation để bảo vệ dữ liệu riêng tư. Các thuộc tính của các lớp này thường được khai báo là private hoặc protected, chỉ có thể truy cập thông qua các phương thức getter và setter công khai. Việc này đảm bảo rằng dữ liệu chỉ được truy cập và thay đổi theo các quy tắc được xác định bởi lớp.

Các lớp trong package gui (như GameCanvas, PlayerDrawer, HelpPage, MainMenu, MainWindow) cũng sử dụng tính encapsulation để quản lý và bảo vệ các thành phần giao diện người dùng. Các thành phần này được khai báo là private hoặc protected và chỉ được truy cập thông qua các phương thức công khai. Điều này giúp giới hạn truy cập và xử lý các thành phần GUI chỉ thông qua các phương thức được quy định trước.

Các lớp Board, Player và Game có các phương thức công khai để truy cập và thay đổi dữ liệu trong các đối tượng tương ứng. Ví dụ: Board có phương thức getCells() để truy cập danh sách các ô, Player có các phương thức getPoint() và setPoint() để truy cập và thiết lập điểm số của người chơi.

Tổng cộng, tính encapsulation trong đoạn code trên đảm bảo tính riêng tư và bảo mật của dữ liệu và cung cấp các phương thức công khai để truy cập và tương tác với các đối tượng theo cách được quy định bởi các lớp tương ứng.

Abstraction (trừu tượng)

In the above code, the abstraction is shown through the use of classes and methods to hide the implementation details and focus on the abstract and important concepts of the game O An Quan. Here are some examples of the use of abstraction in code:

The Player, Board, and Game classes represent the abstract concepts in the game Umbrella. They contain properties and methods related to core aspects of the game such as player scores, the board, and rules of play. These classes provide an abstract interface for interacting with game elements without knowing the internal implementation details.

Interface Board is used to define common methods related to the chessboard and provides an abstract interface for managing and accessing tiles on the board. Using interfaces helps to isolate interfaces from concrete classes to easily replace or extend chessboard classes.

The GameCanvas, PlayerDrawer, HelpPage, MainMenu, and MainWindow classes define the user interfaces and provide abstract functions for interacting with the user. They hide the details of the graphics-related settings and create an abstract interface to display the board, players, menus, and help pages. Using these classes separates the user interface from the game logic.

To summarize, the abstraction in the above code is demonstrated through the use of classes, methods, and interfaces to create abstractions and hide implementation details. This helps to focus on the important concepts of the game, without having to worry about the details inside.

------------------------------------------------------------------------------------

Trong đoạn code trên, tính abstraction (trừu tượng) được thể hiện qua việc sử dụng các lớp và phương thức để ẩn đi các chi tiết cài đặt và tập trung vào các khái niệm trừu tượng và quan trọng của trò chơi Ô Ăn Quan. Dưới đây là một số ví dụ về sự sử dụng của abstraction trong code:

Các lớp Player, Board, và Game đại diện cho các khái niệm trừu tượng trong trò chơi Ô Ăn Quan. Chúng chứa các thuộc tính và phương thức liên quan đến các khía cạnh cốt lõi của trò chơi như điểm số của người chơi, bàn cờ và quy tắc chơi. Các lớp này cung cấp một giao diện trừu tượng cho việc tương tác với các yếu tố trò chơi mà không cần biết chi tiết cài đặt bên trong.

Interface Board được sử dụng để định nghĩa các phương thức chung liên quan đến bàn cờ và cung cấp một giao diện trừu tượng cho việc quản lý và truy cập các ô trên bàn cờ. Việc sử dụng interface giúp tách biệt giao diện với các lớp cụ thể để dễ dàng thay thế hoặc mở rộng các lớp bàn cờ.

Các lớp GameCanvas, PlayerDrawer, HelpPage, MainMenu, và MainWindow định nghĩa các giao diện người dùng và cung cấp các chức năng trừu tượng để tương tác với người dùng. Chúng ẩn đi các chi tiết cài đặt liên quan đến đồ họa và tạo ra một giao diện trừu tượng để hiển thị bàn cờ, người chơi, menu và trang trợ giúp. Việc sử dụng các lớp này giúp tách biệt phần giao diện người dùng và phần logic của trò chơi.

Tóm lại, abstraction trong đoạn code trên được thể hiện thông qua việc sử dụng các lớp, phương thức và interface để tạo ra các khái niệm trừu tượng và ẩn đi các chi tiết cài đặt. Điều này giúp tập trung vào các khái niệm quan trọng của trò chơi Ô Ăn Quan mà không phải quan tâm đến chi tiết bên trong.

Đa hình polymophism:

In the above code, polymorphism is used in several different respects. Here are some examples of how polymorphism is applied in code:

- Polymorphism through inheritance: Classes like Player, Board, and Game inherit from their respective superclasses and provide overridden methods to implement in their own way. For example, the Player1 and Player2 subclasses inherit from Player and implement the makeMove() method to perform player moves one and two. This allows to call the makeMove() method on the Player object without knowing the specific subclass being used.

- Polymorphism through the use of interfaces: The Board interface defines common methods for managing and accessing tiles on the chessboard. The Board and BoardDrawer classes implement this interface and provide concrete implementations for drawing and managing the chessboard. This allows the use of Board and BoardDrawer objects without regard to how exactly the methods are implemented.

- Polymorphism via parametric polymorphism: In the drawStones() method of the BoardDrawer class, the parameter g is of type Graphics2D. This type is a subclass of Graphics and allows the use of special methods available only in Graphics2D. By using parametric polymorphism, the drawStones() method can take a Graphics or Graphics2D object and work properly.

In a nutshell, polymorphism is used in the above code through inheritance, using interfaces and parametric polymorphism. This helps to create flexibility and extensibility in the use of classes and methods, allowing for efficient abstraction and reuse of code.

----------------------------------------------------------------------------------------

Trong đoạn mã trên, đa hình được sử dụng trong một số khía cạnh khác nhau. Dưới đây là một số ví dụ về cách đa hình được áp dụng trong mã:

Đa hình thông qua việc kế thừa: Các lớp như Player, Board, và Game kế thừa từ các lớp cha tương ứng và cung cấp các phương thức được ghi đè (override) để triển khai theo cách riêng của mình. Ví dụ, các lớp con Player1 và Player2 kế thừa từ Player và triển khai phương thức makeMove() để thực hiện nước đi của người chơi một và hai. Điều này cho phép gọi phương thức makeMove() trên đối tượng Player mà không cần biết lớp con cụ thể đang được sử dụng.

Đa hình thông qua sử dụng giao diện (interface): Giao diện Board định nghĩa các phương thức chung cho việc quản lý và truy cập vào ô trên bàn cờ. Các lớp Board và BoardDrawer triển khai giao diện này và cung cấp các triển khai cụ thể cho việc vẽ và quản lý bàn cờ. Điều này cho phép sử dụng các đối tượng Board và BoardDrawer mà không cần quan tâm đến cách chính xác các phương thức được triển khai.

Đa hình thông qua đa hình tham số: Trong phương thức drawStones() của lớp BoardDrawer, tham số g có kiểu Graphics2D. Kiểu này là một lớp con của Graphics và cho phép sử dụng các phương thức đặc biệt chỉ có sẵn trong Graphics2D. Bằng cách sử dụng đa hình tham số, phương thức drawStones() có thể nhận đối tượng Graphics hoặc Graphics2D và hoạt động đúng.

Tóm lại, đa hình được sử dụng trong đoạn mã trên thông qua kế thừa, sử dụng giao diện và đa hình tham số. Điều này giúp tạo ra tính linh hoạt và mở rộng trong việc sử dụng các lớp và phương thức, cho phép trừu tượng hóa và tái sử dụng mã một cách hiệu quả.

# The me code 



## Main Menu

This code represents a class called `MainMenu` which extends `JPanel` and is part of a graphical user interface (GUI) package. It provides the main menu functionality for a game application.

Let's go through the code step by step:

1. The code begins with importing the necessary classes from the AWT, Swing, and IO packages.
```java
import java.awt.*;
import java.awt.event.*;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import javax.imageio.ImageIO;
import javax.swing.*;
``` 

2. The `MainMenu` class is defined as a subclass of `JPanel`. This means that it inherits all the features and behaviors of `JPanel`.
```java
public class MainMenu extends JPanel{
```

3. The class has several instance variables declared at the beginning:
```java
private JButton start;
private JButton help;
private JButton exit;
private JLabel title;
private MainWindow pa;
private BackGroundDrawer bg;
```
These variables are used to create and manage various GUI components such as buttons, labels, and the main window.

4. The constructor of the `MainMenu` class is defined as follows:
```java
public MainMenu(MainWindow pa) {
    this.pa = pa;
    this.bg = new BackGroundDrawer(this);
    
    // Code to create and configure GUI components
    
    // Code to add components to the panel using GridBagLayout
    
}
```
The constructor takes a parameter `pa` of type `MainWindow`, which is used to pass the reference of the parent window to the `MainMenu` object. Inside the constructor, it initializes the instance variables `pa` and `bg`, creating a new `BackGroundDrawer` object.

5. Within the constructor, various GUI components (i.e., `JLabel` and `JButton`) are created and configured:
```java
title = new JLabel("Ô Ăn Quan", JLabel.CENTER);
title.setFont(new Font("SansSerif", Font.BOLD, 60));

start = new JButton("New game");
help = new JButton("Help");
exit = new JButton("Exit");
```
Here, a `JLabel` named `title` is created with the text "Ô Ăn Quan" and centered alignment. The font of the `title` label is set to "SansSerif", bold style, and size 60. Then, three `JButton` objects are created with their respective labels.

6. Three `ActionListener` objects are defined using anonymous inner classes:
```java
ActionListener startGame = new ActionListener() {
    @Override
    public void actionPerformed(ActionEvent e) {
        pa.newGame();
    }
};

ActionListener openHelp = new ActionListener() {
    @Override
    public void actionPerformed(ActionEvent e) {
        pa.showHelp();
    }
};

ActionListener exitGame = new ActionListener() {
    @Override
    public void actionPerformed(ActionEvent e) {
        pa.exitGame();
    }
};
```
These action listeners define what should happen when the buttons are clicked. They call methods (`newGame()`, `showHelp()`, and `exitGame()`) on the parent window object `pa`.

7. The action listeners are added to their respective buttons:
```java
start.addActionListener(startGame);
help.addActionListener(openHelp);
exit.addActionListener(exitGame);
```
This ensures that when the buttons are clicked, the corresponding action listeners will be triggered.

8. The layout manager for the panel is set to `GridBagLayout`:
```java
setLayout(new GridBagLayout());
```

9. The panel's `paintComponent` method is overridden to draw the background:
```java
@Override
protected void paintComponent(Graphics g) {
    super.paintComponent(g);
    bg.setMyG(g);
    bg.draw();
}
```
In this method, the `paintComponent` method of the superclass is called first to ensure the panel is properly painted. Then, the `setMyG` method of the `bg` object is called to set the graphics object for drawing. Finally, the `draw` method of the `bg` object is invoked to draw the background.

That's it! This code creates a main menu panel with buttons and a title label, and provides functionality to handle button clicks through action listeners.

## MainWindow 

This code represents a class called `MainWindow` which extends `JFrame` and is part of a GUI package. It provides the main window functionality for a game application.

Let's go through the code step by step:

1. The code begins with importing the necessary classes from the Swing and AWT packages.
```java
import javax.swing.*;
import javax.swing.JOptionPane;
import java.awt.*;
import controls.*;
```

2. The `MainWindow` class is defined as a subclass of `JFrame`. This means that it inherits all the features and behaviors of `JFrame`.
```java
public class MainWindow extends JFrame{
```

3. The class has several instance variables declared at the beginning:
```java
private GameCanvas myGC;
private MainMenu myMenu;
private HelpPage myHelp;
private int mode;
private Game myGame;
private CardLayout c;
```
These variables are used to create and manage various components such as canvas, menu, help page, game object, and layout manager.

4. The constructor of the `MainWindow` class is defined as follows:
```java
public MainWindow(Game g) {
    this.myGame = g;
    this.mode = 1;
    myGC = new GameCanvas(this.myGame, this);
    myMenu = new MainMenu(this);
    myHelp = new HelpPage(this);
    c = new CardLayout();
    setLayout(c);
    
    // Code to add components to the frame using CardLayout
    
    setVisible(true);
    setSize(800, 600);
    setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
    setTitle("Ô Ăn Quan");
    setLocationRelativeTo(null);
}
```
The constructor takes a parameter `g` of type `Game`, which is used to pass the reference of the game object to the `MainWindow` object. Inside the constructor, it initializes the instance variables `myGame`, `myGC`, `myMenu`, `myHelp`, and `c`.

5. Within the constructor, various components (i.e., `GameCanvas`, `MainMenu`, `HelpPage`) are created and configured:
```java
myGC = new GameCanvas(this.myGame, this);
myMenu = new MainMenu(this);
myHelp = new HelpPage(this);
c = new CardLayout();
setLayout(c);
```
Here, a `GameCanvas` object named `myGC` is created with the game object and the main window as parameters. Similarly, a `MainMenu` object named `myMenu` and a `HelpPage` object named `myHelp` are created with the main window as a parameter. Then, a `CardLayout` object `c` is created and set as the layout manager for the frame.

6. The components (`myMenu`, `myGC`, `myHelp`) are added to the frame using the `add` method of the `JFrame` class:
```java
add("Menu", myMenu);
add("GC", myGC);
add("Help", myHelp);
```
These statements add the components to the frame using the specified names as identifiers.

7. Other methods are defined within the `MainWindow` class, such as `playGame()`, `newGame()`, `showHelp()`, `backToMain()`, `exitGame()`, `redraw()`, and `getMode()`. These methods provide functionality to play the game, start a new game, show help, go back to the main menu, exit the game, redraw the canvas, and retrieve the current mode respectively.

8. In the `exitGame()` method, a confirmation dialog is displayed using `JOptionPane` to ask the user if they want to quit the game. If the user selects "Yes", then the system is exited with `System.exit(0)`.

9. The `redraw()` method is used to repaint the canvas by calling the `repaint()` method on the `myGC` object.

That's it! This code creates a main window for the game application, with components such as canvas, menu, and help page. It also provides methods to handle game play, new game, help, going back to the main menu, and exiting the game.

## HelpPage

The `HelpPage` class represents a panel that displays the help information for the game. Here's a breakdown of the code:

1. The class extends `JPanel`, meaning it inherits all the features and behaviors of `JPanel`.
```java
public class HelpPage extends JPanel{
```

2. The class has several instance variables declared at the beginning:
```java
private MainWindow pa;
private BackGroundDrawer bg;
private JPanel pane;
```
These variables are used to reference the main window, handle background drawing, and create a panel for displaying content.

3. The constructor of the `HelpPage` class is defined as follows:
```java
public HelpPage(MainWindow m) {
    this.pa = m;
    this.bg = new BackGroundDrawer(this);
    // ...
}
```
The constructor takes a parameter `m` of type `MainWindow` and assigns it to the `pa` variable. It also creates a `BackGroundDrawer` object named `bg` with the help page as a parameter.

4. Inside the constructor, the help content is created using a `JLabel` and formatted as HTML:
```java
JLabel content = new JLabel("<html>"
        + "<h1>How to play</h1>"
        + "<p>"
        + "Hover the cursor over one of five square in your side.<br>"
        + "The square will turn blue if it's a valid move.<br>"
        + "An arrow will appear depends on whether the cursor is to the left or the right of the square, indicating the direction of the move.<br>"
        + "You click on the square to make your move.<br>"
        + "The game ends when both big cells are emptied.<br>"
        + "</p>"
        + "<h1>Hướng dẫn chơi game</h1>"
        + "<p>"
        + "Đưa chuột lên một trong năm ô thuộc về bên của bạn.<br>"
        + "Ô đó sẽ chuyển màu xanh dương.<br>"
        + "Nếu bạn đưa chuột về bên phải ô sẽ hiện lên ▶, nếu đưa chuột về bên trái ô sẽ hiện lên ◀, tương ứng với chiều rải quân.<br>"
        + "Bạn click chuột để thực hiện nước đi của mình.<br>"
        + "Trò chơi kết thúc khi hai ô quan đã được ăn hết.<br>"
        + "</html>");
content.setBorder(new EmptyBorder(50, 50, 50, 50));
```
This `JLabel` holds the text content and is formatted using HTML tags. The content provides instructions for how to play the game in both English and Vietnamese.

5. A back button is created using a `JButton`:
```java
JButton back = new JButton("Back");
// ...
```
The button is labeled "Back". 

6. An ActionListener is defined to handle the button's action:
```java
ActionListener goBack = new ActionListener() {
    @Override
    public void actionPerformed(ActionEvent e) {
        pa.backToMain();
    }
};
back.addActionListener(goBack);
```
When the button is clicked, it calls the `backToMain()` method of the `MainWindow` object referenced by `pa`.

7. The `content` label and the `back` button are aligned within a panel named `pane`:
```java
pane = new JPanel();
pane.setLayout(new BoxLayout(pane, BoxLayout.Y_AXIS));
pane.add(content);
pane.add(back);
pane.setOpaque(false);
```
The `BoxLayout` is used to align the components vertically. The `setOpaque(false)` method is called to make sure the panel is transparent.

8. The `pane` panel is added to the `HelpPage` object:
```java
setLayout(new FlowLayout());
add(pane);
```

9. The `paintComponent()` method is overridden to draw the background using a `BackGroundDrawer` object:
```java
@Override
public void paintComponent(Graphics g) {
    super.paintComponent(g);
    
    this.bg.setMyG(g);
    this.bg.draw();
}
```
The `BackGroundDrawer` object is assigned the `Graphics` object and used to draw the background.

That's it! This code creates a `HelpPage` panel that displays instructions for playing the game in multiple languages and provides a back button to return to the main menu.

## MainProgram
The `MainProgram` class represents the entry point of the program. Here's a breakdown of the code:

1. The necessary imports are included at the beginning:
```java
import controls.*;
import gui.MainWindow;
```

2. The `MainProgram` class has a static `main` method, which is the starting point of the program:
```java
public static void main(String[] args) {
    // ...
}
```

3. Within the `main` method, a `Game` object named `myGame` is created:
```java
Game myGame = new Game();
```
This creates an instance of the `Game` class.

4. A `MainWindow` object named `myWindow` is created with the `myGame` object as a parameter:
```java
MainWindow myWindow = new MainWindow(myGame);
```
This creates an instance of the `MainWindow` class and passes the `myGame` object to it.

5. A `while` loop is used to continuously run the game:
```java
while (true) {
    // ...
}
```

6. Inside the `while` loop, there is another `while` loop that runs as long as the game mode in the `myWindow` object is equal to 2:
```java
while (myWindow.getMode() == 2) {
    myWindow.redraw();
    myWindow.playGame();
}
```
This loop redraws the window and plays the game as long as the game mode is set to 2.

7. After the inner `while` loop ends, there is a `for` loop that pauses the program for a certain amount of time before continuing:
```java
for (int i = 0; i < 1000000; i++) {}
```
This loop simply loops a large number of times, causing a delay in the program execution.

8. The program continues back to the beginning of the outer `while` loop, and the process repeats indefinitely.

That's it! This code creates a game instance and a main window, and then continuously runs the game while the game mode is set to 2.